In [1]:
import urllib
import urllib.request
import urllib.parse
from urllib.request import urlopen 
from urllib.request import urlretrieve 

from datetime import datetime
import time,locale

from bs4 import BeautifulSoup
import re
import unicodedata

import pandas as pd

### Create empty dataframes for all classes

In [5]:
tournament_fields = ['name', 'club_id','start_date','url','lat', 'long','price_1_tab','price_2_tabs','price_3_tabs']
dfTourn = pd.DataFrame(columns = tournament_fields)
dfTourn.index.names = ['id']

tournament_ranking_fields = ['name']
dfTournament_ranking = pd.DataFrame(columns = tournament_ranking_fields)
dfTournament_ranking.index.names = ['id']


tournament_category_fields = ['name']
dfTournament_category = pd.DataFrame(columns = tournament_category_fields)
dfTournament_category.index.names = ['id']


player_fields = ['license','name','surname','gender']
dfPlayer = pd.DataFrame(columns = player_fields)


accepts_ranking_fields = ['tid','trid']
dfAccepts_ranking = pd.DataFrame(columns = accepts_ranking_fields)


accepts_category_fields = ['tid','tcid']
dfAccepts_category = pd.DataFrame(columns = accepts_category_fields)


participates_fields = ['pid','tid']
dfParticipates = pd.DataFrame(columns = participates_fields)

### create table tournament_ranking

In [6]:
dfTournament_ranking.loc[0] = ['N1']
dfTournament_ranking.loc[1] = ['N2']
dfTournament_ranking.loc[2] = ['N3']
dfTournament_ranking.loc[3] = ['R4']
dfTournament_ranking.loc[4] = ['R5']
dfTournament_ranking.loc[5] = ['R6']
dfTournament_ranking.loc[6] = ['D7']
dfTournament_ranking.loc[7] = ['D8']
dfTournament_ranking.loc[8] = ['D9']
dfTournament_ranking.loc[9] = ['P10']
dfTournament_ranking.loc[10] = ['P11']
dfTournament_ranking.loc[11] = ['P12']
dfTournament_ranking.loc[12] = ['NC']
dfTournament_ranking.loc[13] = ['undefined']

dfTournament_ranking.to_csv('TournamentRanking.csv')

### method that, given a list of rankings, finds the corresponding ids and adds the pairs to table accepts_ranking

In [9]:
def addRanking(rk, tid):
    switcher = {
        'D' : [6, 7, 8],
        'D+' : [6, 7],
        'D-' : [7, 8],
        'D7' : [6],
        'D8' : [7],
        'D9' : [8],
        'N' : [0, 1, 2],
        'N+' : [0, 1 ],
        'N-' : [1, 2],
        'N1' : [0],
        'N2' : [1],
        'N3' : [2],
        'NC' : [12],
        'P' : [9, 10, 11],
        'P+' : [9, 10],
        'P-' : [10, 11],
        'P10' : [9],
        'P11' : [10],
        'P12' : [11],
        'R' : [3, 4, 5],
        'R+' : [3, 4],
        'R-' : [4, 5],
        'R4' : [3],
        'R5' : [4],
        'R6 ' : [5]
    }
    for r in rk:
        gets = switcher.get(r, [13])
        for g in gets: 
            dfAccepts_ranking.loc[len(dfAccepts_ranking)] = [tid, g]
        

### create table tournament_category

In [15]:
dfTournament_category.loc[0] = ['Veteran']
dfTournament_category.loc[1] = ['Senior']
dfTournament_category.loc[2] = ['Jeune']
dfTournament_category.loc[3] = ['Undefined']

dfTournament_category.to_csv('TournamentCategory.csv')

### method that, given a list of categories, finds the corresponding id and adds the pairs to table accepts_category

In [11]:
def addCategory(cat, tid):
    switcher = {
        'Veteran' : 0,
        'Vétéran' : 0,
        'Vet' : 0,
        'Vét' : 0,
        'Senior' : 1,
        'Jeune' : 2
    }
    for r in cat:
        get = switcher.get(r, [3])
        dfAccepts_category.loc[len(dfAccepts_category)] = [tid, get]

### fetch page 

In [12]:
pages = []

for i in range(8):
    url = 'http://badiste.fr/liste-tournoi-badminton?c2=0&u2=1&page2=' + str(i + 1) + '#table2'
    data = urllib.parse.urlencode({'date' : '100'}).encode("utf-8")

    req = urllib.request.Request(url)
    with urllib.request.urlopen(req,data=data) as f:
        resp = f.read()

    #codeSource = html.read().decode('utf-8')
    soup = BeautifulSoup(resp, 'html.parser')
    pages.append(soup)

### Store clubs in dataframe

In [19]:
dfClub = pd.read_csv('badiste_Club.csv', encoding = 'iso-8859-15', sep=';')
dfClub.drop(columns = dfClub.columns[range(4,17)], inplace=True)
dfClub.columns = ['name','short_name','dep','city_name']
dfClub.drop(columns='dep', inplace=True)
dfClub.index.names = ['id']

dfClub["name_search"] = dfClub["name"] + " (" + dfClub["short_name"] + ")"
dfClub.to_csv('Club.csv', columns = ['name','short_name','city_name','lat','long','url'])
def getClubId(search_str, df):
    return df.index[df['name_search'] == search_str].tolist()


### Fetch all tournaments of one page

In [21]:
pageNb = 1
for p in pages:
    print("scraping page number ", pageNb)
    pageNb = pageNb + 1
    trs = p.find_all("tr", {"class": ["tux", "tux1"]})
    for i in range(len(trs)):
        
        if ((i * 100 / len(trs))%10 == 0):
            print("percentage: ", (i * 100 / len(trs)))
        
        tourn = trs[i] #TODO: change 15 into i
        tds = tourn.find_all('td')

        names = tds[1].find_all('a')[1]
        link = names['href']
        name = names.text.replace('17-18', '')

        cat = tds[7].text
        cats = re.split(' et |,|',cat)
        addCategory(cats, i)

        classmt = tds[8].text
        clas = re.split(',|;|/|-',classmt)   
        addRanking(clas, i)

        html = urlopen("http://badiste.fr/" + link)
        codeSource = html.read().decode('iso-8859-15')
        tsoup = BeautifulSoup(codeSource, 'html.parser')

        infos = tsoup.find_all("td", {"class":  "forminfo"})

        clubName = infos[0].text
        clubId = getClubId(clubName, dfClub)

        realDate = infos[1].text
        locale.setlocale(locale.LC_TIME, "fr_FR") 
        datetime_object = datetime.strptime(realDate, '%A %d %B %Y')
        date = datetime_object.strftime("%d.%m.%Y")

        cost = infos[8].text
        c = re.search('[^0-9]*(\d*)[^0-9]*(\d*)[^0-9]*(\d*).*', str(cost))

        try:
            onetb = c.group(1)
        except IndexError:
            onetb = ""

        try:
            twotb = c.group(2)
        except IndexError:
            twotb = ""
        try:
            threetb = c.group(3)
        except IndexError:
            threetb = ""

        geo = infos[9]
        m = re.search('GPS : (-?\d*.\d*), (-?\d*.\d*)', str(geo))
        if m:
            lat = m.group(1)
            long = m.group(2)

        dfTourn.loc[len(dfTourn)] = [name, clubId[0] if len(clubId) > 0 else -1,  date, link, lat, long, onetb, twotb, threetb]

        time.sleep(0.5)

scraping page number  1
percentage:  0.0
percentage:  10.0
percentage:  20.0
percentage:  30.0
percentage:  40.0
percentage:  50.0
percentage:  60.0
percentage:  70.0
percentage:  80.0
percentage:  90.0
scraping page number  2
percentage:  0.0
percentage:  10.0
percentage:  20.0
percentage:  30.0
percentage:  40.0
percentage:  50.0
percentage:  60.0
percentage:  70.0
percentage:  80.0
percentage:  90.0
scraping page number  3
percentage:  0.0
percentage:  10.0
percentage:  20.0
percentage:  30.0
percentage:  40.0
percentage:  50.0
percentage:  60.0
percentage:  70.0
percentage:  80.0
percentage:  90.0
scraping page number  4
percentage:  0.0
percentage:  10.0
percentage:  20.0
percentage:  30.0
percentage:  40.0
percentage:  50.0
percentage:  60.0
percentage:  70.0
percentage:  80.0
percentage:  90.0
scraping page number  5
percentage:  0.0
percentage:  10.0
percentage:  20.0
percentage:  30.0
percentage:  40.0
percentage:  50.0
percentage:  60.0
percentage:  70.0
percentage:  80.0
per

In [22]:

dfTourn.to_csv('Tournament.csv')

dfAccepts_ranking.to_csv('accepts_ranking.csv', index = False)

dfAccepts_category.to_csv('accepts_category.csv', index = False)

#dfParticipates = pd.DataFrame(columns = participates_fields)
#not done participate yet


### fetch all players for each club and store them in df

In [ ]:
c = 0
wrongClubs = []
for c in range(len(dfClub) - 1): #to remove the fédération
#for c in range(3):
    print("Club n° "+ str(c) +"/" + str(len(dfClub)))
    clubFullName = dfClub.at[c, 'name']
    sigle = dfClub.at[c, 'short_name']
    lowName = unicodedata.normalize('NFD', sigle).encode('ascii', 'ignore').decode('utf-8').lower()
    if(not ('comite' in lowName or 'ligue' in lowName)):
        c = c + 1
        try: 
            playersLink = 'http://badiste.fr/rechercher-joueur-badminton?todo=search&nom=&prenom=&licence=&genre=0&classement=0&categorie=0&ligue=-1&ville=&sigle='+ sigle +'&lim=1000&Submit=Rechercher'

            dataClub = urllib.parse.urlencode({'date' : '100'}).encode("utf-8")

            reqClub = urllib.request.Request(playersLink)
            with urllib.request.urlopen(reqClub,data=dataClub) as f:
                respClub = f.read()

            #codeSource = html.read().decode('utf-8')
            soupClub = BeautifulSoup(respClub, 'html.parser')
            genders = {'H': 0, 'F': 1}
            trsClub = soupClub.find_all("tr", {"class": "tux"})
            pl = 0
            player_count = 0
            for player in trsClub:
                
                if ((player_count * 100 / len(trsClub))%10 == 0):
                    print("\tplayers: " + (player_count * 100 / len(trsClub)) + "%")
                player_count+=1
                
                tdsClub = player.find_all('td')

                nameClub = re.search('(.[^A-Z]+)+([A-Z ]+)', tdsClub[0].text)
                playerName = nameClub.group(1).strip()
                playerSurname = nameClub.group(2).strip()

                if(not 'anonyme' in playerName.lower()):
                    playerLicense = tdsClub[10].text

                    playerGender = genders[tdsClub[1].text]

                    dfPlayer.loc[pl] = [playerLicense, playerName, playerSurname, playerGender]
                pl = pl + 1
        except:
            wrongClubs.append(sigle)
        time.sleep(0.5)
dfPlayer.to_csv('Player.csv', index=False)
wrongDf = pd.DataFrame({'col':wrongClubs})
wrongDf.to_csv('wrong_club_names.csv', index=False)

percentage:  0.0
